In [8]:
import os
from pprint import pprint
import pandas as pd

os.chdir("/home/jovyan/work/BERT")
import bert

In [9]:
pwd

'/home/jovyan/work/BERT'

In [10]:
doc_file = open("../data/unstructure_data/text.txt")
doc_file = doc_file.read()

In [11]:
names_string = "Hi there Jon Jones Jon Jones Jr. Jon Paul Jones John D. Jones,"
ner_list = bert.identify(doc_file)

In [6]:
pprint(ner_list)

[{'end': 5,
  'entity': 'B-PER',
  'index': 1,
  'score': 0.9994486,
  'start': 0,
  'word': 'Sarah'},
 {'end': 13,
  'entity': 'I-PER',
  'index': 2,
  'score': 0.99925476,
  'start': 6,
  'word': 'Johnson'},
 {'end': 40,
  'entity': 'B-LOC',
  'index': 7,
  'score': 0.99939096,
  'start': 31,
  'word': 'Amsterdam'},
 {'end': 53,
  'entity': 'B-LOC',
  'index': 9,
  'score': 0.99964,
  'start': 42,
  'word': 'Netherlands'},
 {'end': 61,
  'entity': 'B-MISC',
  'index': 11,
  'score': 0.61826694,
  'start': 59,
  'word': 'BS'},
 {'end': 118,
  'entity': 'B-PER',
  'index': 27,
  'score': 0.7437089,
  'start': 116,
  'word': 'sa'},
 {'end': 125,
  'entity': 'I-PER',
  'index': 31,
  'score': 0.50708586,
  'start': 123,
  'word': '##oh'},
 {'end': 129,
  'entity': 'B-PER',
  'index': 32,
  'score': 0.589739,
  'start': 125,
  'word': '##nson'},
 {'end': 523,
  'entity': 'B-MISC',
  'index': 112,
  'score': 0.8151688,
  'start': 520,
  'word': 'GDP'},
 {'end': 581,
  'entity': 'B-PER',
  

In [7]:
df = pd.DataFrame(ner_list, columns=['entity', 'word', 'start', 'end', 'score', 'index'])

print(df)

    entity         word  start   end     score  index
0    B-PER        Sarah      0     5  0.999449      1
1    I-PER      Johnson      6    13  0.999255      2
2    B-LOC    Amsterdam     31    40  0.999391      7
3    B-LOC  Netherlands     42    53  0.999640      9
4   B-MISC           BS     59    61  0.618267     11
5    B-PER           sa    116   118  0.743709     27
6    I-PER         ##oh    123   125  0.507086     31
7    B-PER       ##nson    125   129  0.589739     32
8   B-MISC          GDP    520   523  0.815169    112
9    B-PER        Sarah    576   581  0.999469    121
10   I-PER      Johnson    582   589  0.999144    122
11  B-MISC          GDP    764   767  0.800077    152
12  B-MISC          GDP    789   792  0.737091    158
13  B-MISC           BS    954   956  0.749784    190
14  B-MISC          GDP   1049  1052  0.592432    206
15   B-ORG     European   1131  1139  0.996458    221
16   I-ORG        Union   1140  1145  0.995735    222
17   B-ORG           EU   12

In [26]:
this_name = []
all_names_list_tmp = []

for ner_dict in ner_list:
    if ner_dict['entity'] == 'B-PER':
        if len(this_name) == 0:
            this_name.append(ner_dict['word'])
        else:
            all_names_list_tmp.append([this_name])
            this_name = []
            this_name.append(ner_dict['word'])
    elif ner_dict['entity'] == 'I-PER':
        this_name.append(ner_dict['word'])

all_names_list_tmp.append([this_name])

pprint(all_names_list_tmp)

[[['Sarah', 'Johnson']], [['sa', '##oh']], [['##nson']], [['Sarah', 'Johnson']]]


In [27]:
final_name_list = []
for name_list in all_names_list_tmp:
    full_name = ' '.join(name_list[0]).replace(' ##', '').replace(' .', '.')
    final_name_list.append([full_name])

pprint(final_name_list)

[['Sarah Johnson'], ['saoh'], ['##nson'], ['Sarah Johnson']]
